In [1]:
# Question and Problem Definition
# can our model determine based on a given test dataset if passengers in the test dataset survived or not

In [2]:
# Based on https://www.kaggle.com/startupsci/titanic-data-science-solutions

In [3]:
# Workflow goals
# 1. Classifying
# 2. Correlating
# 3. Coverting
# 4. Completing
# 5. Correcting
# 6. Creating
# 7. Charting

In [4]:
import pandas as pd
import numpy as np
import random as rnd

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [5]:
train_df = pd.read_csv('../input/titanic/train.csv')
test_df = pd.read_csv('../input/titanic/test.csv')
combine = [train_df, test_df]

In [6]:
train_df.columns

In [7]:
# Categorical features:
    # Categorical: Survived, Sex, and Embarked
    # Ordinal: Pclass
# Numerical features:
    # Continuous: Age, Fare
    # Discrete: SibSp, Parch

In [8]:
train_df.head()

In [9]:
train_df.tail()

In [10]:
# Mixed data types
    # Ticket and Cabin
# May contain errors or typos
    # Name

In [11]:
train_df.info()
test_df.info()

In [12]:
# Features that contain blank, empty, or null values
    # Age, cabin, embarked in training
    # Age, cabin in test

In [13]:
# Exploring distribution of numerical feature values

In [14]:
train_df.describe()

In [15]:
sns.histplot(train_df.Survived, stat='density')

In [16]:
# about 38% of the samples survived

In [17]:
sns.histplot(train_df.Parch, bins=np.arange(0, 7, 1), stat='density')

In [18]:
# more than 75% of samples did not travel with parents or children

In [19]:
sns.histplot(train_df.SibSp, bins=np.arange(0, 9, 1), stat='density')

In [20]:
# about 32% of samples traveled with spouses or siblings

In [21]:
sns.histplot(train_df.Fare, bins=np.arange(0, 600, 50), stat='density')

In [22]:
# a few samples paid significantly higher fares

In [23]:
sns.histplot(train_df.Age, stat='density')

In [24]:
# a few elderly samples, mostly 20 - 40

In [25]:
# Exploring distribution of categorical feature values

In [26]:
train_df.describe(include=['O'])
# include variables that have string values

In [27]:
# names are unique
# 65% male
# tickets have high duplicate values
# cabins are shared
# 3 embarked ports, S is the most common port

In [28]:
# Assumptions based on data analysis
    # Correlating
        # do correlations early in the project
        # compare with modeled correlations later
    # Completing
        # complete Age and Embarked as they're likely to be correlated with Survival
    # Correcting
        # drop Ticket as it's unlikely to be correlated with Survival and has high duplicate values
        # drop Cabin since it has a lot of null values 
        # drop PassengerId in the training dataset
        # drop or change Name
    # Creating
        # we may create a new feature called Family that includes the number of family members on board
        # we may extract Title from Name
        # we may create AgeBands that's an ordinal categorical feature
        # we may create FareRange if it helps
    # Classifying
        # We may also add to our assumptions based on the problem description
            # 1. Women (Sex=female) were more likely to have survived
            # 2. Children (Age<?) were more likely to have survived
            # 3. The upper-class passengers (Pclass=1) were more likely to have survived

In [29]:
# Analyze by pivoting (turning) features to confirm assumptions and observations
    # we can only work on data without empty values
    # we work on categorical (sex), ordinal (Pclass), and discrete (SibSp, Parch) type

In [30]:
train_df.get(['Pclass', 'Survived']).groupby('Pclass').mean().sort_values(by='Survived', ascending=False)

In [31]:
# correlation between Pclass and Survived, assumption #3, include Pclass in model

In [32]:
train_df.get(['Sex', 'Survived']).groupby('Sex').mean().sort_values(by='Survived', ascending=False)

In [33]:
# correlation between Sex and Survived, assumption #1, include Pclass in model

In [34]:
train_df.get(['SibSp', 'Survived']).groupby('SibSp').mean().sort_values(by='Survived', ascending=False)

In [35]:
# SibSp has 0 correlation for certain values

In [36]:
train_df.get(['Parch', 'Survived']).groupby('Parch').mean().sort_values(by='Survived', ascending=False)

In [37]:
# Parch has 0 correlation for certain values
# we can create a feature from Sibsp and Parch, create #1

In [38]:
# Analyze by visualizing data

In [39]:
# Correlating numerical features

In [40]:
g = sns.FacetGrid(train_df, col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [41]:
# infants have high survival rate
# oldest passengers survived
# large number of 15-25 year olds did not survive
# most passengers are in 15-35 range

# include Age in our model, assumption #2
# complete Age, complete #1
# band Age groups, create #3

In [42]:
# Correlating numerical and ordinal features

In [43]:
g = sns.FacetGrid(train_df, row='Pclass', col='Survived')
g.map(plt.hist, 'Age', bins=20)

In [44]:
# Pclass 3 has the most passengers, but most did not survive, assumption #2
# infant passengers in Pclass 2 and 3 mostly survived, assumption #2
# most passengers in Pclass 1 survived, assumption #3
# Pclass varies in terms of Age distributions

# include Pclass in model

In [45]:
# Correlating categorical features

In [46]:
g = sns.FacetGrid(train_df, row='Embarked')
g.map(sns.pointplot, 'Pclass', 'Survived', 'Sex')
g.add_legend()

In [47]:
# females had a higher survival rate (Exception in Embarked C - could be a correlation between Embarked and Pclass), assumption #1
# some indications of the relationship between Pclass and Survived, revealed by embarked, completing #2
# varying survival rates for Pclass = 3 and among male passengers, correlatin #1

# include Sex in the model,
# complete and include Embarked

In [48]:
# Correlating categorical and numerical features

In [49]:
g = sns.FacetGrid(train_df, row='Embarked', col='Survived')
g.map(sns.barplot, 'Sex', 'Fare')

In [50]:
# passengers who paid more had a higher survival rate, creating #4
# embarked ports correlate with survival rates, correlating #1 and completing #2

# banding fare

In [51]:
# Wrangle data

In [52]:
# dropping features
train_df = train_df.drop(['Ticket', 'Cabin'], axis=1)
test_df = test_df.drop(['Ticket', 'Cabin'], axis=1)

In [53]:
combine = [train_df, test_df]

In [54]:
# creating new features
# create Title by extracting information from Name
for dataset in combine:
    dataset['Title'] = dataset.Name.str.extract('([A-Za-z]+)\.', expand=False)

In [55]:
pd.crosstab(train_df.Title, train_df.Sex)

In [56]:
# replace some titles with more common names
for dataset in combine:
    dataset['Title'] = dataset['Title'].replace(['Lady', 'Countess','Capt', 'Col', 'Don', 'Dr', 'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona'], 'Rare')
    dataset['Title'] = dataset['Title'].replace('Mlle', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Ms', 'Miss')
    dataset['Title'] = dataset['Title'].replace('Mme', 'Mrs')
    
train_df.get(['Title', 'Survived']).groupby('Title').mean()

In [57]:
# convert categorical variables to ordinal
title_mapping = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
for dataset in combine:
    dataset['Title'] = dataset['Title'].map(title_mapping)
    dataset['Title'] = dataset['Title'].fillna(0)

train_df.head()

In [58]:
# drop Name and PassengerId
train_df = train_df.drop(['Name', 'PassengerId'], axis=1)
test_df = test_df.drop(['Name'], axis=1)
combine = [train_df, test_df]

In [59]:
# Converting a categorical feature

In [60]:
for dataset in combine:
    dataset['Sex'] = dataset['Sex'].map( {'female': 1, 'male': 0} ).astype(int)

train_df.head()

In [61]:
# Completing a numerical continuous feature

In [62]:
# for Age, we guess Age values using median values for Age across sets of Pclass and Gender feature combinations

In [63]:
guess_ages = np.zeros((2, 3))
guess_ages

In [64]:
for dataset in combine:
    for i in range(0, 2):
        for j in range(0, 3):
            guess_df = dataset[(dataset['Sex'] == i) & (dataset['Pclass'] == j+1)]['Age'].dropna()
            age_guess = guess_df.median()
            # Convert random age float to nearest .5 age
            guess_ages[i,j] = int(age_guess/0.5 + 0.5) * 0.5
    for i in range(0, 2):
        for j in range(0, 3):
            dataset.loc[ (dataset.Age.isnull()) & (dataset.Sex == i) & (dataset.Pclass == j+1),'Age'] = guess_ages[i,j]

    dataset['Age'] = dataset['Age'].astype(int)

train_df.head()

In [65]:
# create age bands and determine their correlations with Survived
train_df['AgeBand'] = pd.cut(train_df['Age'], 5)
train_df.get(['AgeBand', 'Survived']).groupby('AgeBand').mean().sort_values(by='AgeBand', ascending=True)

In [66]:
# replace Age with ordinals based on these bands
for dataset in combine:    
    dataset.loc[ dataset['Age'] <= 16, 'Age'] = 0
    dataset.loc[(dataset['Age'] > 16) & (dataset['Age'] <= 32), 'Age'] = 1
    dataset.loc[(dataset['Age'] > 32) & (dataset['Age'] <= 48), 'Age'] = 2
    dataset.loc[(dataset['Age'] > 48) & (dataset['Age'] <= 64), 'Age'] = 3
    dataset.loc[ dataset['Age'] > 64, 'Age']
train_df.head()

In [67]:
train_df = train_df.drop(['AgeBand'], axis=1)
combine = [train_df, test_df]
train_df.head()

In [68]:
# Create new feature combining existing features

In [69]:
# create FamilySize that combines Parch and SibSp
for dataset in combine:
    dataset['FamilySize'] = dataset['SibSp'] + dataset['Parch'] + 1

train_df.get(['FamilySize', 'Survived']).groupby('FamilySize').mean().sort_values(by='Survived', ascending=False)

In [70]:
# create IsAlone
for dataset in combine:
    dataset['IsAlone'] = 0
    dataset.loc[dataset['FamilySize'] == 1, 'IsAlone'] = 1

train_df.get(['IsAlone', 'Survived']).groupby('IsAlone').mean()

In [71]:
# drop Parch, SibSp, and FamilySize features in favor of IsAlone
train_df = train_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
test_df = test_df.drop(['Parch', 'SibSp', 'FamilySize'], axis=1)
combine = [train_df, test_df]

train_df.head()

In [72]:
# create an artificial feature combining Pclass and Age
for dataset in combine:
    dataset['Age*Class'] = dataset.Age * dataset.Pclass

train_df.loc[:, ['Age*Class', 'Age', 'Pclass']].head(10)

In [73]:
# Completing a categorical feature

In [74]:
# fill two null values for Embarked using the mode
freq_port = train_df.Embarked.dropna().mode()[0]
freq_port

In [75]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].fillna(freq_port)
    
train_df.get(['Embarked', 'Survived']).groupby('Embarked').mean().sort_values(by='Survived', ascending=False)

In [76]:
# convert categorical feature to numeric

In [77]:
for dataset in combine:
    dataset['Embarked'] = dataset['Embarked'].map( {'S': 0, 'C': 1, 'Q': 2} ).astype(int)

train_df.head()

In [78]:
# Quick completing and converting a numeric feature

In [79]:
# complete the Fare feature for single missing value in test dataset using median
# round off to two decimals since it's currency
test_df['Fare'].fillna(test_df['Fare'].dropna().median(), inplace=True)
test_df.head()

In [80]:
# create FareBand
train_df['FareBand'] = pd.qcut(train_df['Fare'], 4)
train_df.get(['FareBand', 'Survived']).groupby('FareBand').mean().sort_values(by='FareBand', ascending=True)

In [81]:
for dataset in combine:
    dataset.loc[ dataset['Fare'] <= 7.91, 'Fare'] = 0
    dataset.loc[(dataset['Fare'] > 7.91) & (dataset['Fare'] <= 14.454), 'Fare'] = 1
    dataset.loc[(dataset['Fare'] > 14.454) & (dataset['Fare'] <= 31), 'Fare']   = 2
    dataset.loc[ dataset['Fare'] > 31, 'Fare'] = 3
    dataset['Fare'] = dataset['Fare'].astype(int)

train_df = train_df.drop(['FareBand'], axis=1)
combine = [train_df, test_df]
    
train_df.head(10)

In [82]:
test_df.head(10)

In [83]:
# Model, predict, and solve

In [84]:
# separate the feature to be predicted from the training dataset
X_train = train_df.drop('Survived', axis=1).copy()
Y_train = train_df['Survived']
X_test = test_df.drop('PassengerId', axis=1).copy()
X_train.shape, Y_train.shape, X_test.shape

In [85]:
# Logistic Regression

In [86]:
logreg = LogisticRegression()
logreg.fit(X_train, Y_train)
Y_pred = logreg.predict(X_test)
acc_log = round(logreg.score(X_train, Y_train) * 100, 2)
acc_log

In [87]:
# use the coefficients to validate assumptions and decisions
coeff_df = pd.DataFrame(train_df.columns.delete(0))
coeff_df.columns = ['Feature']
coeff_df["Correlation"] = pd.Series(logreg.coef_[0])
coeff_df.sort_values(by='Correlation', ascending=False)

In [88]:
# Sex is the highest positive coefficient
# Title is the second highest positive coefficient
# Pclass is the lowest negative coefficient
# Age*Class is the second lowest negative coefficient -> good artificial feature

In [89]:
# Support Vector Machines

In [90]:
svc = SVC()
svc.fit(X_train, Y_train)
Y_pred = svc.predict(X_test)
acc_svc = round(svc.score(X_train, Y_train) * 100, 2)
acc_svc

In [91]:
# K Nearest Neighbors

In [92]:
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(X_train, Y_train)
Y_predict = knn.predict(X_test)
acc_knn = round(knn.score(X_train, Y_train) * 100, 2)
acc_knn

In [93]:
# Decision Tree

In [94]:
decision_tree = DecisionTreeClassifier()
decision_tree.fit(X_train, Y_train)
Y_predict = decision_tree.predict(X_test)
acc_decision_tree = round(decision_tree.score(X_train, Y_train) * 100, 2)
acc_decision_tree

In [95]:
# Random Forest

In [96]:
random_forest = RandomForestClassifier(n_estimators=100)
random_forest.fit(X_train, Y_train)
Y_predict = random_forest.predict(X_test)
acc_random_forest = round(random_forest.score(X_train, Y_train) * 100, 2)
acc_random_forest

In [97]:
# Model evaluations

In [98]:
models = pd.DataFrame({'Model': ['LogReg', 'SVM', 'KNN', 'DecisionTree', 'RandomForest'], 'Score': [acc_log, acc_svc, acc_knn, acc_decision_tree, acc_random_forest]})
models.sort_values(by='Score', ascending=False)

In [99]:
# Submission

In [100]:
submission = pd.DataFrame({
    'PassengerId': test_df['PassengerId'],
    'Survived': Y_predict
})
# submission.to_csv('../output/submission.csv', index=False)